# Beating the Zestimate® | Predicting Home Values
Stephanie N. Jones<br>
Data Scientist<br>
Codeup | Hopper Cohort<br>
Monday, December 13, 2021

## Project Summary
<hr>

### Project Overview
This is an overview of the project.

### Deliverables
    - Final Report (this notebook) and Presentation
    - Github Repository with README.md and reproducible project files
    - Python Function Modules (acquire.py and prepare.py, at minimum)
>Data Dictionary

## Business Goals
<hr>

### Primary Goal #1 | Predict Home Values
Construct Machine Learning model by finding key drivers that predicts property tax assessed values <b>`taxvaluedollarcnt`</b> of Single Family Properties with transactions during 2017

### Primary Goal #2 | Beat Zestimate®
Improve existing Zillow model for predicting home values 

### Secondary Goal | Property Locations
City and state of each property

## Executive Summary
<hr>

### Initial Questions and Hypothesis
>Question 1?<br>
Question 2?<br>
Question 3?<br>
Question 4?<br>

>Initial Hypothesis

### Conclusions
>Key findings

>Recommendations

>Takeaways and Next Steps

## Process | Data Science Pipeline
<hr>

00 Plan<br>
01 Acquire<br>
02 Prepare<br>
03 Explore<br>
04 Model<br>
05 Evaluate<br>
06 Test

### 00 Plan

### 01 Acquire

### 02 Prepare

### 03 Explore

### 04 Model

### 05 Evaluate

### 06 Test

## Conclusions
<hr>

### Key Findings
>These were my key findings.

### Recommendations
>These are my recommendations based on those key findings.

### Takeaways
>Here are my final takeaways from the project and, if I had more time, these would be my next steps.